In [2]:
import json

car_group = dict()


k5 = dict()

k5["price"] = "5000"

k5["year"] = "2015"

car_group["K5"] = k5


avante = dict()

avante["price"] = "3000"

avante["year"] = "2014"

car_group["Avante"] = avante


#json 파일로 저장

with open('./test.json', 'w', encoding='utf-8') as make_file:

    json.dump(car_group, make_file, indent="\t")


# 저장한 파일 출력하기

with open('./test.json', 'r') as f:

    json_data = json.load(f)

print(json.dumps(json_data, indent="\t"))


{
	"K5": {
		"price": "5000",
		"year": "2015"
	},
	"Avante": {
		"price": "3000",
		"year": "2014"
	}
}


In [22]:
import fitz
import PIL.Image
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar,LTLine,LAParams
import os
import json
import PIL.Image
import io
import re


path = r'./os.pdf'
extract_data = {}
pdf = fitz.open(path)
page_id = 1
cur_size = 0
text = ""

# full-text -> image 순으로 페이지당으로 나옴
for i in range(len(pdf)):
    each_page = {}
    page = pdf[i]  # load page
    images = page.get_images()
    text = page.get_text('sentences')
    
    # for image in images:
    #     base_img = pdf.extract_image(image[0])
    #     image_data = base_img['image']
    #     img = PIL.Image.open(io.BytesIO(image_data))
    #     extension = base_img['ext']
    #     img.save(
    #         open(f"temp_dataset/split_pages/image{page_id}.{extension}", "wb"))
    each_page["page_id"] = page_id
    full_text_data = {"audio_url": "", "full_text": text}
    each_page["full_text"] = full_text_data
    extract_data[str(page_id)] = each_page
    page_id += 1

# line-text 페이지당으로 나옴       
for page_layout in extract_pages(path):
    each_page = {}
    info = []
    font_size = 0
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            for text_line in element:
                for character in text_line:
                    if isinstance(character, LTChar):
                        next_size = round(character.size)
                if (cur_size != 0) or (cur_size != next_size):
                    cur_size = next_size
                    text = ""
                if cur_size == next_size:
                    text += element.get_text()
                info.append(
                    {"audio_url": "", "font_size": cur_size, "text": text})
            
    extract_data[str(page_layout.pageid)]["text"] = info

# 정확도 높이기
# 줄바꿈 기준 쪼개고 글씨크기 기준으로 정확히 나누기
for i in range(1, len(extract_data) + 1):
    each_page_size = len(extract_data[str(i)]["text"])
    each_page = extract_data[str(i)]["text"]
    for j in range(each_page_size - 1):
        cur_text = each_page[j]["text"]
        next_text = each_page[j + 1]["text"]
        if cur_text == next_text:
            split_list = each_page[j]["text"].split("\n")
            info = []
            for k in range(len(split_list)):
                font_size = each_page[j + k]["font_size"]
                text = split_list[k] + "\n"
                text = re.sub(r"[^\w\s]]", "", text)  # import re
                if text == "\n":
                    continue
                else:
                    each_page[j + k] = {"audio_url": "",
                                        "font_size": font_size, "text": text}
# with open('./split.json', 'w', encoding='utf-8') as make_file:
#     json.dump(extract_data, make_file, indent="\t", ensure_ascii=False)
# print("fin")
concat_text = ""
# 글씨 크기 같은 애들은 묶기
for i in range(1, len(extract_data) + 1):
    each_page_size = len(extract_data[str(i)]["text"])
    each_page = extract_data[str(i)]["text"]
    concat_text = each_page[0]["text"]
    to_del_list = []
    for j in range(each_page_size - 1):
        # j = 0
        # while j != each_page_size - 1:
        cur_size = each_page[j]["font_size"]
        next_size = each_page[j + 1]["font_size"]
        if cur_size == next_size:
            concat_text += each_page[j + 1]["text"]
            to_del_list.append(j)
            # del(each_page[j + 1])
        else:
            each_page[j]["text"] = concat_text
            concat_text = each_page[j + 1]["text"]
            # to_del_list.append(j - 1)
            j += 1
    # print(to_del_list)
    # 반복문을 통해서 리스트 뒤에서부터 지우기
    # 스택이나 큐를 사용해서 제거
    list_len = len(to_del_list)
    for j in range(list_len - 1, -1, -1):
        del(each_page[to_del_list[j]])
    #     print(to_del_list[j])
    # print()


# print(extract_data)
with open('./concat.json', 'w', encoding='utf-8') as make_file:
    json.dump(extract_data, make_file, indent="\t", ensure_ascii=False)
print("fin")


fin


In [21]:
import re

with open('./sample_text_only.json', 'r', encoding="utf-8") as f:

    extract_data = json.load(f)

# print(json.dumps(json_data, indent="\t", ensure_ascii=False))

# 정확도 높이기
# 줄바꿈 기준 쪼개고 글씨크기 기준으로 정확히 나누기
for i in range(1, len(extract_data) + 1):
    each_page_size = len(extract_data[str(i)]["text"])
    each_page = extract_data[str(i)]["text"]
    for j in range(each_page_size - 1):
        cur_text = each_page[j]["text"]
        next_text = each_page[j + 1]["text"]
        if cur_text == next_text:
            split_list = each_page[j]["text"].split("\n")
            info = []
            for k in range(len(split_list)):
                font_size = each_page[j + k]["font_size"]
                text = split_list[k] + "\n"
                text = re.sub(r"[^\w\s]]", "", text) # import re
                if text == "\n":
                    continue
                else:
                    each_page[j + k] = {"audio_url":"", "font_size": font_size, "text": text}
# with open('./split.json', 'w', encoding='utf-8') as make_file:
#     json.dump(extract_data, make_file, indent="\t", ensure_ascii=False)
# print("fin")
concat_text = ""        
# 글씨 크기 같은 애들은 묶기
for i in range(1, len(extract_data) + 1):
    each_page_size = len(extract_data[str(i)]["text"])
    each_page = extract_data[str(i)]["text"]
    concat_text = each_page[0]["text"]
    to_del_list = []
    for j in range(each_page_size - 1):
    # j = 0
    # while j != each_page_size - 1:
        cur_size = each_page[j]["font_size"]
        next_size = each_page[j + 1]["font_size"]
        if cur_size == next_size:
            concat_text += each_page[j + 1]["text"]
            to_del_list.append(j)
            # del(each_page[j + 1])
        else:
            each_page[j]["text"] = concat_text
            concat_text = each_page[j + 1]["text"]
            # to_del_list.append(j - 1)
            j += 1
    # print(to_del_list)
    # 반복문을 통해서 리스트 뒤에서부터 지우기
    # 스택이나 큐를 사용해서 제거
    list_len = len(to_del_list)
    for j in range(list_len - 1, -1, -1):
        del(each_page[to_del_list[j]])
    #     print(to_del_list[j])
    # print()
        
    
# print(extract_data)
with open('./concat.json', 'w', encoding='utf-8') as make_file:
    json.dump(extract_data, make_file, indent="\t", ensure_ascii=False)
print("fin")


fin


In [11]:
temp = [1, 2, 3, 4, 5, 6]
for i in range(len(temp), 0, -1):
    print(i)

6
5
4
3
2
1
